# import packages

In [2]:
import os
import torch
import pandas as pd
from transformers import BertModel, DistilBertModel
# import kobert_tokenizer

In [3]:
os.getcwd()

'C:\\Users\\Administrator\\Desktop\\Work\\2. 인공지능리서치AIR\\1. source code'

## save pre-trained kobert model (2021.07.30)

In [3]:
# torch.save(model, 'pretrained_kobert_20210730.pth')

# data sets

In [4]:
data_path = "C:/Users/Administrator/Desktop/Work/2. 인공지능리서치AIR/2. data sets/1. input data"
file_name1 = "(1차)13,500건.xlsx"
os.path.join(data_path, file_name1)
data_set1 = pd.read_excel(os.path.join(data_path, file_name1))

In [5]:
data_set1.shape

(13500, 9)

In [6]:
file_name2 = "(2차)34,500건.xlsx"
os.path.join(data_path, file_name2)
data_set2 = pd.read_excel(os.path.join(data_path, file_name2))

In [7]:
data_set2.shape

(34500, 9)

In [8]:
data_set = data_set1.append(data_set2)

In [9]:
data_set.shape

(48000, 9)

In [10]:
data_set.columns = ['uid',
                   'publisher',
                   'title',
                   'summary',
                   'content',
                   'content_url',
                    'update_at',
                   'importance',
                   'polarity']

In [11]:
data_set.head(1)

,uid,publisher,title,summary,content,content_url,update_at,importance,polarity
0,354652739944452421,뉴스1,김진석 CJ헬로비전 대표 '주주총회 70%이상 찬성으로 합병안 가결',임세영 기자 = 김진석 CJ헬로비전 대표가 26일 오전 서울 마포구 상암동 누리꿈스...,임세영 기자 = 김진석 CJ헬로비전 대표가 26일 오전 서울 마포구 상암동 누리꿈스...,http://news.naver.com/main/read.nhn?mode=LSD&m...,2016-02-26T10:25:00,+,+


In [12]:
data_set.to_excel('C:/Users/Administrator/Desktop/Work/2. 인공지능리서치AIR/2. data sets/1. input data/merged_data_set_AIR.xlsx')

In [11]:
data_set_cleaned1 = data_set[data_set.importance.notnull()]
# data_set_cleaned1.shape
data_set_cleaned1_2 = data_set_cleaned1[data_set_cleaned1.polarity.notnull()]
data_set_cleaned2 = data_set_cleaned1_2[data_set_cleaned1_2.importance != '0']
# data_set_cleaned2.shape
data_set_cleaned2['importance'].value_counts()

-    23854
+    14656
Name: importance, dtype: int64

In [12]:
data_set_cleaned2.head(1)

,uid,publisher,title,summary,content,content_url,update_at,importance,polarity
0,354652739944452421,뉴스1,김진석 CJ헬로비전 대표 '주주총회 70%이상 찬성으로 합병안 가결',임세영 기자 = 김진석 CJ헬로비전 대표가 26일 오전 서울 마포구 상암동 누리꿈스...,임세영 기자 = 김진석 CJ헬로비전 대표가 26일 오전 서울 마포구 상암동 누리꿈스...,http://news.naver.com/main/read.nhn?mode=LSD&m...,2016-02-26T10:25:00,+,+


## Labeling

In [15]:
from sklearn.preprocessing import LabelEncoder

In [16]:
encoder = LabelEncoder()

# importance
encoder.fit(data_set_cleaned2['importance'])
data_set_cleaned2['importance_int'] = encoder.transform(data_set_cleaned2['importance'])
# polarity
encoder.fit(data_set_cleaned2['polarity'])
data_set_cleaned2['polarity_int'] = encoder.transform(data_set_cleaned2['polarity'])

<ipython-input-16-7b66ed1a4174>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_set_cleaned2['importance_int'] = encoder.transform(data_set_cleaned2['importance'])
<ipython-input-16-7b66ed1a4174>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_set_cleaned2['polarity_int'] = encoder.transform(data_set_cleaned2['polarity'])


In [17]:
mapping = dict(zip(range(len(encoder.classes_)), encoder.classes_))
mapping

{0: '+', 1: '-', 2: '0'}

In [18]:
data_set_cleaned2['importance'].value_counts()

-    23854
+    14656
Name: importance, dtype: int64

In [171]:
data_set_cleaned2['importance_int'] = data_set_cleaned2['importance_int'].astype(np.int64)

<ipython-input-171-88e4bed54ae8>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_set_cleaned2['importance_int']=data_set_cleaned2['importance_int'].astype(np.int64)


In [172]:
data_set_cleaned2['importance_int']

0        0
2        1
3        1
4        1
7        0
        ..
34495    1
34496    1
34497    1
34498    1
34499    1
Name: importance_int, Length: 38510, dtype: int64

In [132]:
data_set_cleaned2['importance_int'].value_counts()

1    23854
0    14656
Name: importance_int, dtype: int64

In [18]:
data_set_cleaned2['polarity'].value_counts()

0    25776
+    14702
-     6751
Name: polarity, dtype: int64

In [19]:
data_set_cleaned2['polarity_int'].value_counts()

2    25776
0    14702
1     6751
Name: polarity_int, dtype: int64

# 1 Binary classifier 

### https://zzaebok.github.io/deep_learning/nlp/Bert-for-classification/

In [20]:
# pip install pytorch-transformers

In [66]:
import numpy as np
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from pytorch_transformers import BertTokenizer, BertForSequenceClassification, BertConfig
from torch.optim import Adam
import torch.nn.functional as F

In [176]:
data_set_cleaned2.head(1)

,uid,publisher,title,summary,content,content_url,update_at,importance,polarity,importance_int,polarity_int
0,354652739944452421,뉴스1,김진석 CJ헬로비전 대표 '주주총회 70%이상 찬성으로 합병안 가결',임세영 기자 = 김진석 CJ헬로비전 대표가 26일 오전 서울 마포구 상암동 누리꿈스...,임세영 기자 = 김진석 CJ헬로비전 대표가 26일 오전 서울 마포구 상암동 누리꿈스...,http://news.naver.com/main/read.nhn?mode=LSD&m...,2016-02-26T10:25:00,+,+,0,0


In [177]:
new_data = data_set_cleaned2[['content', 'importance_int']]

In [178]:
new_data.shape

(38510, 2)

In [179]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(new_data, test_size=0.2, random_state=42)
print("train shape is:", len(train))
print("test shape is:", len(test))

train shape is: 30808
test shape is: 7702


In [215]:
class AirDataset(Dataset):
    ''' Dataset '''
    def __init__(self, df):
        self.df = df

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        text = self.df.iloc[idx, 0]
        label = self.df.iloc[idx, 1]
        return text, label

In [72]:
Air_train_dataset = AirDataset(train[:20])
train_loader = DataLoader(Air_train_dataset,
                         batch_size = 3,
                         shuffle = True,
                         num_workers = 0)

In [217]:
# tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
# model = BertForSequenceClassification.from_pretrained('bert-base-multilingual-cased',
#                                                      num_labels = 3)

In [73]:
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
model = BertForSequenceClassification.from_pretrained('bert-base-multilingual-cased',
                                                     num_labels = 3)

In [ ]:
# encoded_list = [tokenizer.encode(t, add_special_tokens=True) for t in text]

## 1-1. Train

In [68]:
optimizer = Adam(model.parameters(), lr=1e-6)

itr = 500
p_itr = 500
epochs = 1
total_loss = 0
total_len = 0
total_correct = 0


model.train()
for epoch in range(epochs):
    
    for text, label in train_loader:
        optimizer.zero_grad()
        
        # encoding and zero padding
        encoded_list = [tokenizer.encode(t, add_special_tokens=True) for t in text]
        # padded_list =  [e + [0] * (119547-len(e)) for e in encoded_list]
        padded_list =  [e[:min(len(e), 512)] + [0] * (512-len(e[:min(512, len(e))])) for e in encoded_list]
        
        sample = torch.tensor(padded_list)
        # sample, label = sample.to(device), label.to(device)
        # sample, label = sample.to(device), label.to(device)
        labels = torch.tensor(label)
        outputs = model(sample, labels=labels)
        loss, logits = outputs

        pred = torch.argmax(F.softmax(logits), dim=1)
        correct = pred.eq(labels)
        total_correct += correct.sum().item()
        total_len += len(labels)
        total_loss += loss.item()
        loss.backward()
        optimizer.step()
        
        if itr % p_itr == 0:
            print('[Epoch {}/{}] Iteration {} -> Train Loss: {:.4f}, Accuracy: {:.3f}'.format(epoch+1, epochs, itr, total_loss/p_itr, total_correct/total_len))
            total_loss = 0
            total_len = 0
            total_correct = 0

        itr+=1

Token indices sequence length is longer than the specified maximum sequence length for this model (758 > 512). Running this sequence through the model will result in indexing errors
Token indices sequence length is longer than the specified maximum sequence length for this model (1049 > 512). Running this sequence through the model will result in indexing errors


AttributeError: 'float' object has no attribute 'split'

## 1-2. Test

In [ ]:
# evaluation
model.eval()

nsmc_eval_dataset = NsmcDataset(test_df)
eval_loader = DataLoader(nsmc_eval_dataset, batch_size=2, shuffle=False, num_workers=2)

total_loss = 0
total_len = 0
total_correct = 0

for text, label in eval_loader:
    encoded_list = [tokenizer.encode(t, add_special_tokens=True) for t in text]
    padded_list =  [e + [0] * (512-len(e)) for e in encoded_list]
    sample = torch.tensor(padded_list)
    sample, label = sample.to(device), label.to(device)
    labels = torch.tensor(label)
    outputs = model(sample, labels=labels)
    _, logits = outputs

    pred = torch.argmax(F.softmax(logits), dim=1)
    correct = pred.eq(labels)
    total_correct += correct.sum().item()
    total_len += len(labels)

print('Test accuracy: ', total_correct / total_len)

# 2 Multi class classification with BERT

### https://towardsdatascience.com/multi-class-text-classification-with-deep-learning-using-bert-b59ca2f5c613

In [13]:
import torch
from tqdm.notebook import tqdm

from transformers import BertTokenizer
from torch.utils.data import TensorDataset

from transformers import BertForSequenceClassification

## Data

In [14]:
data_set_cleaned1_2.head(1)

,uid,publisher,title,summary,content,content_url,update_at,importance,polarity
0,354652739944452421,뉴스1,김진석 CJ헬로비전 대표 '주주총회 70%이상 찬성으로 합병안 가결',임세영 기자 = 김진석 CJ헬로비전 대표가 26일 오전 서울 마포구 상암동 누리꿈스...,임세영 기자 = 김진석 CJ헬로비전 대표가 26일 오전 서울 마포구 상암동 누리꿈스...,http://news.naver.com/main/read.nhn?mode=LSD&m...,2016-02-26T10:25:00,+,+


In [15]:
data_set_cleaned1_2['importance'].value_counts()

-    23854
+    14656
0     8719
Name: importance, dtype: int64

In [16]:
possible_labels = data_set_cleaned1_2.importance.unique()
possible_labels

array(['+', '0', '-'], dtype=object)

### Encoding the labels

In [17]:
possible_labels = data_set_cleaned1_2.importance.unique()

label_dict ={}
for index, possible_label in enumerate(possible_labels):
    label_dict[possible_label] = index
# label_dict['+'] = 1
# label_dict['-'] = -1
# label_dict['0'] = 0
label_dict

{'+': 0, '0': 1, '-': 2}

In [18]:
data_set_cleaned1_2['label'] = data_set_cleaned1_2.importance.replace(label_dict)
data_set_cleaned1_2['label'].value_counts()

<ipython-input-18-2aba9da0d81b>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_set_cleaned1_2['label'] = data_set_cleaned1_2.importance.replace(label_dict)


2    23854
0    14656
1     8719
Name: label, dtype: int64

In [19]:
from sklearn.model_selection import train_test_split
# df = data_set_cleaned1_2['content', 'importance','label']
df = data_set_cleaned1_2
# df = data_set_cleaned1_2.drop(['uid',
#                               'publisher',
#                               'title',
#                               'summary',
#                               'content_url',
#                               'update_at',
#                               'polarity',
#                               'importance_int'], axis = 1)
X_train, X_val, y_train, y_val = train_test_split(df.index.values, 
                                                  df.label.values, 
                                                  test_size = 0.15, 
                                                  random_state = 42, 
                                                  stratify = df.label.values)

df['data_type'] = ['not_set'] * df.shape[0]

df.loc[X_train, 'data_type'] = 'train'
df.loc[X_val, 'data_type'] = 'val'

df.groupby(['importance', 'label', 'data_type']).count()

<ipython-input-19-f21e5db34fea>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['data_type'] = ['not_set'] * df.shape[0]
C:\Users\Administrator\Anaconda3\lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


uid  publisher  title  summary  content  \
importance label data_type                                              
+          0     train      11339      11339  11339    11290    11291   
                 val         3317       3317   3317     3300     3300   
-          2     train      18615      18615  18615    18435    18434   
                 val         5239       5239   5239     5190     5192   
0          1     train       6888       6888   6888     6871     6871   
                 val         1831       1831   1831     1825     1825   

                            content_url  update_at  polarity  
importance label data_type                                    
+          0     train            11339      11339     11339  
                 val               3317       3317      3317  
-          2     train            18615      18615     18615  
                 val               5239       5239      5239  
0          1     train             6888       6888      6888  
                 val               1831       1831      1831

In [20]:
from kobert_transformers import get_tokenizer
# tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
tokenizer = get_tokenizer()

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BertTokenizer'. 
The class this function is called from is 'KoBertTokenizer'.


In [21]:
tokenizer.tokenize("한국어 모델을 공유합니다")

['▁한국', '어', '▁모델', '을', '▁공유', '합니다']

In [24]:
encoded_data_train = tokenizer.batch_encode_plus(
    df[df.data_type == 'train'].title.values, 
    add_special_tokens = True, 
    return_attention_mask = True, 
    pad_to_max_length = True, 
    max_length = 512, 
    return_tensors = 'pt'
)

encoded_data_val = tokenizer.batch_encode_plus(
    df[df.data_type == 'val'].title.values, 
    add_special_tokens = True, 
    return_attention_mask = True, 
    pad_to_max_length = True, 
    max_length = 512, 
    return_tensors = 'pt'
)

In [25]:
input_ids_train = encoded_data_train['input_ids']
attention_masks_train = encoded_data_train['attention_mask']
labels_train = torch.tensor(df[df.data_type == 'train'].label.values)

input_ids_val = encoded_data_val['input_ids']
attention_masks_val = encoded_data_val['attention_mask']
labels_val = torch.tensor(df[df.data_type=='val'].label.values)

In [26]:
dataset_train = TensorDataset(input_ids_train, attention_masks_train, labels_train)
dataset_val = TensorDataset(input_ids_val, attention_masks_val, labels_val)

In [27]:
# model_multiclass = BertForSequenceClassification.from_pretrained('bert-base-multilingual-cased',
#                                                                  num_labels=len(label_dict))
model_multiclass = BertForSequenceClassification.from_pretrained("monologg/kobert",
                                                                        num_labels = 3)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at monologg/kobert and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [28]:
len(label_dict)

3

### Data loader

In [29]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
batch_size = 3
dataloader_train = DataLoader(dataset_train, 
                              sampler = RandomSampler(dataset_train), 
                              batch_size = batch_size)
dataloader_validation = DataLoader(dataset_val, 
                                   sampler = SequentialSampler(dataset_val), 
                                   batch_size = batch_size)

In [30]:
from transformers import AdamW, get_linear_schedule_with_warmup

optimizer = AdamW(model_multiclass.parameters(),
                  lr=1e-5, 
                  eps=1e-8)
                  
epochs = 1

scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps=0,
                                            num_training_steps=len(dataloader_train)*epochs)

### Performance metrics

In [31]:
from sklearn.metrics import f1_score

def f1_score_func(preds, labels):
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return f1_score(labels_flat, preds_flat, average='weighted')

def accuracy_per_class(preds, labels):
    label_dict_inverse = {v: k for k, v in label_dict.items()}
    
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()

    for label in np.unique(labels_flat):
        y_preds = preds_flat[labels_flat==label]
        y_true = labels_flat[labels_flat==label]
        print(f'Class: {label_dict_inverse[label]}')
        print(f'Accuracy: {len(y_preds[y_preds==label])}/{len(y_true)}\n')

### Training loop

In [32]:
import random
import numpy as np

seed_val = 17
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

def evaluate(dataloader_val):

    model_multiclass.eval()
    
    loss_val_total = 0
    predictions, true_vals = [], []
    
    for batch in dataloader_val:
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }

        with torch.no_grad():        
            outputs = model_multiclass(**inputs)
            
        loss = outputs[0]
        logits = outputs[1]
        loss_val_total += loss.item()

        logits = logits.detach().cpu().numpy()
        label_ids = inputs['labels'].cpu().numpy()
        predictions.append(logits)
        true_vals.append(label_ids)
    
    loss_val_avg = loss_val_total/len(dataloader_val) 
    
    predictions = np.concatenate(predictions, axis=0)
    true_vals = np.concatenate(true_vals, axis=0)
            
    return loss_val_avg, predictions, true_vals

In [33]:
os.getcwd()

'C:\\Users\\Administrator\\Desktop\\Work\\2. 인공지능리서치AIR\\1. source code'

In [34]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [35]:
device

device(type='cpu')

In [36]:
epochs = 2

for epoch in tqdm(range(1, epochs+1)):
    model_multiclass.train()
    loss_train_total = 0
    progress_bar = tqdm(dataloader_train, 
                        desc = 'Epoch {:1d}'.format(epoch), 
                        leave=False, 
                        disable=False)
    for batch in progress_bar:
        model_multiclass.zero_grad()
        batch = tuple(b.to(device) for b in batch)
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }       
        outputs = model_multiclass(**inputs)
        
        loss = outputs[0]
        loss_train_total += loss.item()
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model_multiclass.parameters(), 1.0)

        optimizer.step()
        scheduler.step()
        
        progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item()/len(batch))})
         
   #  torch.save(model.state_dict(), 'C:\\Users\\Administrator\\Desktop\\Work\\2. 인공지능리서치AIR\\1. source code\\finetuned_BERT_epoch_{epoch}.model')
        
    tqdm.write(f'\nEpoch {epoch}')
    
    loss_train_avg = loss_train_total/len(dataloader_train)            
    tqdm.write(f'Training loss: {loss_train_avg}')
    
    val_loss, predictions, true_vals = evaluate(dataloader_validation)
    val_f1 = f1_score_func(predictions, true_vals)
    tqdm.write(f'Validation loss: {val_loss}')
    tqdm.write(f'F1 Score (Weighted): {val_f1}')

  0%|          | 0/2 [00:00<?, ?it/s]

Epoch 1:   0%|          | 0/12281 [00:00<?, ?it/s]


Epoch 1
Training loss: 0.8244193436611434
Validation loss: 0.7988650331533467
F1 Score (Weighted): 0.6689007654068125


Epoch 2:   0%|          | 0/12281 [00:00<?, ?it/s]


Epoch 2
Training loss: 0.7134852445781518
Validation loss: 0.7988650331533467
F1 Score (Weighted): 0.6689007654068125


## Loading and Evaluating the model

In [37]:
model

NameError: name 'model' is not defined

In [ ]:
model = BertForSequenceClassification.from_pretrained("bert-base-uncased",
                                                      num_labels=len(label_dict),
                                                      output_attentions=False,
                                                      output_hidden_states=False)

model.to(device)

model.load_state_dict(torch.load('data_volume/finetuned_BERT_epoch_1.model', 
                                 map_location = torch.device('cpu')))

_, predictions, true_vals = evaluate(dataloader_validation)
accuracy_per_class(predictions, true_vals)